<h1><center>Neighborhoods in City of Toronto</center></h1>

<Bold>__Welcome! This notebook is the final Capstone Project for Applied Data Science Capstone Course__</Bold>

### Segmenting and Clustering Neighbourhoods in Toronto

##### Import and install required libraries

In [3]:
!pip install lxml
import pandas as pd # library for data analsysis
import numpy as np

In [ ]:
#Import Pandas, Numpy and Requests libraries
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import json # library to handle JSON files

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

!conda update -n base -c defaults conda -y
#!conda -y install package

print("Libraries imported")


Solving environment: - 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/linux-64::_ipyw_jlab_nb_ext_conf==0.1.0=py37_0
  - defaults/linux-64::alabaster==0.7.11=py37_0
  - defaults/linux-64::anaconda==5.3.1=py37_0
  - defaults/linux-64::anaconda-client==1.7.2=py37_0
  - defaults/linux-64::anaconda-navigator==1.9.2=py37_0
  - defaults/linux-64::anaconda-project==0.8.2=py37_0
  - defaults/linux-64::appdirs==1.4.3=py37h28b3542_0
  - defaults/linux-64::astroid==2.0.4=py37_0
  - defaults/linux-64::astropy==3.0.4=py37h14c3975_0
  - defaults/linux-64::atomicwrites==1.2.1=py37_0
  - defaults/linux-64::automat==0.7.0=py37_0
  - defaults/linux-64::babel==2.6.0=py37_0
  - defaults/linux-64::backports==1.0=py37_1
  - defaults/linux-64::backports.shutil_get_terminal_size==1.0.0=py37_2
  - defaults/linux-64::beautifulsoup4==4.6.3=py37_0
  - defaults/linux-64::bitarray==0.8.3=py37h14c3975_0
  - defaults/linux-

## 1. Download and Explore Dataset

##### Read HTML table to Pandas dataframe

In [5]:
#Read HTML table to pandas dataframe
df_read = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M',header=0)[0]

##### Remove rows with Borough value as 'Not assigned'

In [6]:
#Remove rows with Borough value as 'Not assigned'
df_city = df_read[df_read.Borough != 'Not assigned']

##### Groupby Postcode and Borough and combine Neighbourhood values

In [7]:
#Group the dataframe by Postcode and Borough and join Neighbourhood value with ',' separated values
df_city = df_city.groupby(['Postcode', 'Borough'], as_index=False, sort=False).agg(','.join)

##### Replace Neighbourhood value with Borough value when Neighbouhood is 'Not assigned'

In [8]:
#Replace 'Neighbourhood' value with 'Borough' value when Neighbourhood='Not assigned'
df_city['Neighbourhood'] = np.where((df_city.Neighbourhood == 'Not assigned'), df_city['Borough'], df_city.Neighbourhood)
df_city

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
...,...,...,...
98,M8X,Etobicoke,"The Kingsway,Montgomery Road,Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern
101,M8Y,Etobicoke,"Humber Bay,King's Mill Park,Kingsway Park Sout..."


##### Print the number of rows and columns in the dataframe

In [9]:
print("Number of rows in the dataframe: ", df_city.shape[0])
print("Number of columns in the dataframe: ", df_city.shape[1])

Number of rows in the dataframe:  103
Number of columns in the dataframe:  3


##### Retrieve Latitude and Longitude

In [10]:
df_geo = pd.read_csv('http://cocl.us/Geospatial_data/Geospatial_Coordinates.csv')
print ('Data read into a pandas dataframe!')
df_geo.rename(columns = {'Postal Code':'Postcode'}, inplace = True)
df_geo

Data read into a pandas dataframe!


,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


##### Merge dataframe with city and the dataframe with latitude,longitude

In [11]:
df_comb = pd.merge(df_city, df_geo, how='inner', on = 'Postcode')
df_comb

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway,Montgomery Road,Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558
101,M8Y,Etobicoke,"Humber Bay,King's Mill Park,Kingsway Park Sout...",43.636258,-79.498509


##### Use geopy library to get Latitude and Longitude values of Toronto city¶

In [12]:
address = 'Toronto City'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.653963, -79.387207.


##### Create a map of Toronto City with neighborhoods superimposed on top

In [13]:
df_comb.columns

Index(['Postcode', 'Borough', 'Neighbourhood', 'Latitude', 'Longitude'], dtype='object')

In [16]:
# Create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(df_comb['Latitude'], df_comb['Longitude'], df_comb['Borough'], df_comb['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

##### Segment and cluster only with Boroughs that contain the word Toronto

In [18]:
toronto_data = df_comb[df_comb.Borough.str.contains('Toronto') == True].reset_index(drop=True)
toronto_data

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
1,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
5,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
6,M6G,Downtown Toronto,Christie,43.669542,-79.422564
7,M5H,Downtown Toronto,"Adelaide,King,Richmond",43.650571,-79.384568
8,M6H,West Toronto,"Dovercourt Village,Dufferin",43.669005,-79.442259
9,M5J,Downtown Toronto,"Harbourfront East,Toronto Islands,Union Station",43.640816,-79.381752


##### Explore Toronto neighborhood with Foursquare API

###### Define Foursquare Credentials and version

In [19]:
# @hidden_cell
CLIENT_ID = 'CLIENT_ID' # your Foursquare ID
CLIENT_SECRET = 'CLIENT_SECRET' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

#print('Your credentails:')
#print('CLIENT_ID: ' + CLIENT_ID)
#print('CLIENT_SECRET:' + CLIENT_SECRET)

##### Explore first neighborhood in the dataframe

###### Get the neighborhood's name.

In [20]:
toronto_data.loc[0, 'Neighbourhood']

'Harbourfront,Regent Park'

## 2. Explore Neighborhoods in Toronto

###### Create a function to repeat the same process to all the neighborhoods in Manhattan

###### To simplify the analysis retain only one neighborhood for each Borough and Postcode combination

In [21]:
#df['Column2'] = df.Column2.str.replace('b,?' , '')
#toronto_data['Neighbourhood'] = toronto_data.Neighbourhood.str.replace(',?' , '')
#df['text_new'] = df['text'].str.split('::').str[0]
toronto_data['Neighbourhood'] = toronto_data['Neighbourhood'].str.split(',').str[0]
toronto_data

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
1,M5B,Downtown Toronto,Ryerson,43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
5,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
6,M6G,Downtown Toronto,Christie,43.669542,-79.422564
7,M5H,Downtown Toronto,Adelaide,43.650571,-79.384568
8,M6H,West Toronto,Dovercourt Village,43.669005,-79.442259
9,M5J,Downtown Toronto,Harbourfront East,43.640816,-79.381752


In [22]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&LIMIT={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius,
            LIMIT
            )
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [23]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighbourhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )
toronto_venues

Harbourfront
Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Adelaide
Dovercourt Village
Harbourfront East
Little Portugal
The Danforth West
Design Exchange
Brockton
The Beaches West
Commerce Court
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North
High Park
North Toronto West
The Annex
Parkdale
Davisville
Harbord
Runnymede
Moore Park
Chinatown
Deer Park
CN Tower
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown
First Canadian Place
Church and Wellesley
Business Reply Mail Processing Centre 969 Eastern


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Harbourfront,43.654260,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,Harbourfront,43.654260,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Harbourfront,43.654260,-79.360636,Toronto Cooper Koo Family Cherry St YMCA Centre,43.653191,-79.357947,Gym / Fitness Center
3,Harbourfront,43.654260,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,Harbourfront,43.654260,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
...,...,...,...,...,...,...,...
828,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558,Toronto Yoga Mamas,43.664824,-79.324335,Yoga Studio
829,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558,Olliffe On Queen,43.664503,-79.324768,Butcher
830,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558,TTC Stop #03049,43.664470,-79.325145,Light Rail Station
831,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558,Greenwood Cigar & Variety,43.664538,-79.325379,Smoke Shop


##### Check the size of the resulting venues dataframe

In [24]:
print(toronto_venues.shape)
toronto_venues.head()

(833, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Harbourfront,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,Harbourfront,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Harbourfront,43.65426,-79.360636,Toronto Cooper Koo Family Cherry St YMCA Centre,43.653191,-79.357947,Gym / Fitness Center
3,Harbourfront,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,Harbourfront,43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot


##### Check how many venues were returned for each neighborhood

In [25]:
toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Adelaide,30,30,30,30,30,30
Berczy Park,30,30,30,30,30,30
Brockton,24,24,24,24,24,24
Business Reply Mail Processing Centre 969 Eastern,19,19,19,19,19,19
CN Tower,14,14,14,14,14,14
Cabbagetown,30,30,30,30,30,30
Central Bay Street,30,30,30,30,30,30
Chinatown,30,30,30,30,30,30
Christie,17,17,17,17,17,17


##### Find out how many unique categories can be curated from all the returned venues

In [27]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 188 uniques categories.


## 3. Analyze Each Neighborhood

In [28]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Aquarium,Art Gallery,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


##### Get shape of new dataframe

In [29]:
toronto_onehot.shape

(833, 189)

##### Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [30]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Aquarium,Art Gallery,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,Adelaide,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000
1,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000
2,Brockton,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Business Reply Mail Processing Centre 969 Eastern,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.052632
4,CN Tower,0.071429,0.071429,0.071429,0.142857,0.142857,0.142857,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,Cabbagetown,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,Chinatown,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.066667,0.033333,0.000000
8,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Church and Wellesley,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


##### Get the size of the dataframe

In [31]:
toronto_grouped.shape

(38, 189)

##### Print each neighborhood along with the top 5 most common venues

In [32]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide----
              venue  freq
0              Café  0.10
1             Hotel  0.07
2        Steakhouse  0.07
3  Asian Restaurant  0.07
4         Gastropub  0.03


----Berczy Park----
                venue  freq
0            Beer Bar  0.07
1                Café  0.07
2        Cocktail Bar  0.07
3      Farmers Market  0.07
4  Seafood Restaurant  0.07


----Brockton----
            venue  freq
0  Breakfast Spot  0.08
1            Café  0.08
2          Bakery  0.08
3     Coffee Shop  0.08
4             Gym  0.04


----Business Reply Mail Processing Centre 969 Eastern----
                venue  freq
0  Light Rail Station  0.11
1         Yoga Studio  0.05
2              Garden  0.05
3                Park  0.05
4          Comic Shop  0.05


----CN Tower----
                venue  freq
0      Airport Lounge  0.14
1     Airport Service  0.14
2    Airport Terminal  0.14
3             Airport  0.07
4  Airport Food Court  0.07


----Cabbagetown----
                venue  freq
0  Italia

##### Add it to pandas dataframe

###### Sort venues in descending order

In [33]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

###### Create dataframe and display the top 10 venues for each neighborhood

In [34]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Café,Steakhouse,Hotel,Asian Restaurant,Gastropub,Speakeasy,Smoke Shop,Seafood Restaurant,Burger Joint,Plaza
1,Berczy Park,Seafood Restaurant,Cocktail Bar,Beer Bar,Café,Farmers Market,Bistro,Fish Market,Basketball Stadium,Concert Hall,Jazz Club
2,Brockton,Bakery,Breakfast Spot,Café,Coffee Shop,Grocery Store,Climbing Gym,Sandwich Place,Burrito Place,Restaurant,Caribbean Restaurant
3,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Yoga Studio,Auto Workshop,Smoke Shop,Brewery,Spa,Burrito Place,Farmers Market,Fast Food Restaurant,Restaurant
4,CN Tower,Airport Lounge,Airport Service,Airport Terminal,Airport,Harbor / Marina,Sculpture Garden,Boutique,Bar,Boat or Ferry,Airport Food Court


## 4. Cluster Neighborhoods

#### Run k-means to cluster the neighborhood into 5 clusters.

In [35]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

###### Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [36]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,1,Coffee Shop,Park,Bakery,Breakfast Spot,Mexican Restaurant,Chocolate Shop,Event Space,Dessert Shop,Farmers Market,Spa
1,M5B,Downtown Toronto,Ryerson,43.657162,-79.378937,1,Café,Clothing Store,Steakhouse,Restaurant,Japanese Restaurant,Ramen Restaurant,Sandwich Place,Coffee Shop,Burger Joint,Burrito Place
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,1,Gastropub,Coffee Shop,Italian Restaurant,Restaurant,Hotel,BBQ Joint,Cosmetics Shop,Church,Creperie,Poke Place
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,1,Trail,Health Food Store,Pub,Neighborhood,Yoga Studio,Dance Studio,Ethiopian Restaurant,Eastern European Restaurant,Dumpling Restaurant,Dog Run
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,1,Seafood Restaurant,Cocktail Bar,Beer Bar,Café,Farmers Market,Bistro,Fish Market,Basketball Stadium,Concert Hall,Jazz Club


##### Visualize the resulting clusters

In [37]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examine Clusters

#### Cluster 1

In [38]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
28,Moore Park,Park,Playground,Trail,Tennis Court,Cuban Restaurant,Ethiopian Restaurant,Eastern European Restaurant,Dumpling Restaurant,Dog Run,Discount Store
32,Rosedale,Park,Playground,Trail,Yoga Studio,Dance Studio,Ethiopian Restaurant,Eastern European Restaurant,Dumpling Restaurant,Dog Run,Discount Store


#### Cluster 2

In [39]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Harbourfront,Coffee Shop,Park,Bakery,Breakfast Spot,Mexican Restaurant,Chocolate Shop,Event Space,Dessert Shop,Farmers Market,Spa
1,Ryerson,Café,Clothing Store,Steakhouse,Restaurant,Japanese Restaurant,Ramen Restaurant,Sandwich Place,Coffee Shop,Burger Joint,Burrito Place
2,St. James Town,Gastropub,Coffee Shop,Italian Restaurant,Restaurant,Hotel,BBQ Joint,Cosmetics Shop,Church,Creperie,Poke Place
3,The Beaches,Trail,Health Food Store,Pub,Neighborhood,Yoga Studio,Dance Studio,Ethiopian Restaurant,Eastern European Restaurant,Dumpling Restaurant,Dog Run
4,Berczy Park,Seafood Restaurant,Cocktail Bar,Beer Bar,Café,Farmers Market,Bistro,Fish Market,Basketball Stadium,Concert Hall,Jazz Club
5,Central Bay Street,Coffee Shop,Bubble Tea Shop,Spa,Italian Restaurant,Café,Tea Room,Japanese Restaurant,Bar,Ramen Restaurant,Sushi Restaurant
6,Christie,Grocery Store,Café,Park,Athletics & Sports,Italian Restaurant,Diner,Convenience Store,Nightclub,Candy Store,Restaurant
7,Adelaide,Café,Steakhouse,Hotel,Asian Restaurant,Gastropub,Speakeasy,Smoke Shop,Seafood Restaurant,Burger Joint,Plaza
8,Dovercourt Village,Pharmacy,Supermarket,Bakery,Coffee Shop,Brewery,Gym / Fitness Center,Music Venue,Park,Portuguese Restaurant,Café
9,Harbourfront East,Park,Plaza,Café,Hotel,Performing Arts Venue,Roof Deck,Sporting Goods Shop,Ice Cream Shop,Bistro,Italian Restaurant


#### Cluster 3

In [40]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Lawrence Park,Bus Line,Park,Swim School,Yoga Studio,Event Space,Ethiopian Restaurant,Eastern European Restaurant,Dumpling Restaurant,Dog Run,Discount Store


#### Cluster 4

In [41]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,Forest Hill North,Park,Jewelry Store,Trail,Sushi Restaurant,Yoga Studio,Deli / Bodega,Event Space,Ethiopian Restaurant,Eastern European Restaurant,Dumpling Restaurant


#### Cluster 5

In [42]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Roselawn,Ice Cream Shop,Garden,Music Venue,Dance Studio,Event Space,Ethiopian Restaurant,Eastern European Restaurant,Dumpling Restaurant,Dog Run,Discount Store
